In [ ]:
import re
import json
import tqdm
import random
from datasets import load_dataset
from transformers import AutoTokenizer
random.seed(101)

/home/gcilleru/ProphetNet/GENIE/.venv/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# extract .zip file in recipes/raw/dataset.zip
# !unzip -oq recipes.zip -d recipes

In [2]:
dataset = load_dataset("recipe_nlg", data_dir="recipes/raw/dataset")

Generating train split: 100%|██████████| 2231142/2231142 [06:00<00:00, 6191.54 examples/s]
/home/gcilleru/ProphetNet/GENIE/.venv/lib/python3.8/site-packages/datasets/table.py:1421: FutureWarning: promote has been superseded by mode='default'.
  table = cls._concat_blocks(blocks, axis=0)


In [3]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

In [4]:
def mask_recipe_proportion(recipe, tokenizer, proportion):
    tokens = tokenizer.tokenize(recipe)
    len_recipe = len(tokens)
    max_index = int(len_recipe * (1 - proportion))
    mask_0 = random.randint(0, max_index)
    mask_1 = mask_0 + int(len_recipe * proportion)
    masking30_tgt = tokenizer.decode(tokenizer.convert_tokens_to_ids(tokens[mask_0:mask_1]) )
    # substitute those tokens with [MASK]
    masking30_src = tokenizer.decode(tokenizer.convert_tokens_to_ids(tokens[:mask_0])) + " [MASK] " + tokenizer.decode(tokenizer.convert_tokens_to_ids(tokens[mask_1:]))
    mask = {
        "src_len":len_recipe,
        "tgt_len":len(tokenizer.tokenize(masking30_tgt)),
        "src":masking30_src,
        "tgt":masking30_tgt
    }
    return mask

def is_mask_valid(mask):
    return mask["src_len"] < 512 and mask["tgt_len"] < 128


In [5]:
failed = 0
masked_recipes = []
for sample in tqdm.tqdm(dataset["train"],desc=f"Failed {failed}", total=len(dataset["train"])):
    recipe = sample["title"] + " Ingredients: " + " ".join(sample["ingredients"]) + " Directions: -" + " -".join(sample["directions"])
    # MASKING: 90% of directions, 10% of ingredients
    if random.random()>0.85:
        # mask ingredients
        src = sample["title"] + " Ingredients: " + " [MASK] " + " Directions: -" + " -".join(sample["directions"])
        tgt = " ".join(sample["ingredients"])
        mask_human = {
            "src":src,
            "tgt":tgt,
            "src_len":len(tokenizer.tokenize(src)),
            "tgt_len":len(tokenizer.tokenize(tgt))
        }
    else:
        # sample to mask from 1 to len(directions)-2
        if len(sample["directions"]) > 2:
            num_directions_to_mask = random.randint(1, len(sample["directions"])-2)
            mask_0 = random.randint(0, len(sample["directions"])-num_directions_to_mask)
            mask_1 = mask_0 + num_directions_to_mask
            src_masked_directions = sample["directions"][:mask_0] + ["[MASK]"] + sample["directions"][mask_1:]
            tgt = " -".join(sample["directions"][mask_0:mask_1])
            src = sample["title"] + " Ingredients: " + " ".join(sample["ingredients"]) + " Directions: -" + " -".join(src_masked_directions)
            mask_human = {
                "src":src,
                "tgt":tgt,
                "src_len":len(tokenizer.tokenize(src)),
                "tgt_len":len(tokenizer.tokenize(tgt))
            }

    mask_30 = mask_recipe_proportion(recipe, tokenizer, 0.3)
    if mask_human and is_mask_valid(mask_human) and mask_30 and is_mask_valid(mask_30):
        masked_recipes.append({
            "id":sample["id"],
            "mask_30":mask_30,
            "mask_human":mask_human
        })
    else:
        failed += 1

Failed 0: 100%|██████████| 2231142/2231142 [1:19:29<00:00, 467.84it/s]


In [ ]:
failed

In [ ]:
len(masked_recipes)

In [6]:
# save masked_recipes as json
with open("recipes/raw/masked_recipes.json", "w") as f:
    json.dump(masked_recipes, f)

In [6]:
# load masked_recipes from json
with open("recipes/raw/masked_recipes.json", "r") as f:
    masked_recipes = json.load(f)

NameError: name 'json' is not defined

In [ ]:
masked_recipes[0]

In [7]:
# generate the train.src, train.tgt, valid.src, valid.tgt, test.src, test.tgt
# set seed
random.seed(101)
# split dataset into train, valid, test
random.shuffle(masked_recipes)
train = masked_recipes[:int(len(masked_recipes)*0.8)]
valid = masked_recipes[int(len(masked_recipes)*0.8):int(len(masked_recipes)*0.95)]
test = masked_recipes[int(len(masked_recipes)*0.95):]

for split, recipes in zip(["train", "dev", "test"], [train, valid, test]):
    for folder in ["mask_30", "mask_human"]:
        # recipes/folder/split.
        with open(f"recipes/{folder}/org_data/{split}.src", "w") as f:
            f.write("\n".join([re.sub("\n","",recipe[folder]["src"]) for recipe in recipes]))
        with open(f"recipes/{folder}/org_data/{split}.tgt", "w") as f:
            f.write("\n".join([re.sub("\n","",recipe[folder]["tgt"]) for recipe in recipes]))


In [5]:
len("\n".join([re.sub("\n","",recipe[folder]["src"]) for recipe in train]).split("\n"))

NameError: name 'train' is not defined

In [ ]:
len([re.sub("\n","",recipe[folder]["src"]) for recipe in train])

In [ ]:
len([recipe["mask_human"]["tgt"] for recipe in train])

In [4]:
# count number of line sin each file in recipes/mask_human/org_data
!wc -l recipes/mask_human/org_data/train.src

1576900 recipes/mask_human/org_data/train.src


In [3]:
!wc -l recipes/mask_human/org_data/train.tgt

1576900 recipes/mask_human/org_data/train.tgt


In [10]:
# print first 5 lines from recipes/mask_human/org_data/{split}.src and recipes/mask_human/org_data/{split}.tgt
split = 'train'
with open(f'recipes/mask_human/org_data/{split}.src', 'r') as f:
    src_lines = f.readlines()
with open(f'recipes/mask_human/org_data/{split}.tgt', 'r') as f:
    tgt_lines = f.readlines()
for i in range(5):
    print(f'src: {src_lines[i]}')
    print(f'tgt: {tgt_lines[i]}')
    print()

src: Bean, Pasta and Roasted Pepper Soup Ingredients: 1 tbsp. olive oil 2 large carrots 2 stalk celery 1 large onion 1 tbsp. chopped fresh parsley 3 clove garlic 1 can Swanson Chicken Broth 1 can black beans 1 can diced tomatoes 1/2 c. uncooked ditalini pasta 1/2 c. roasted sweet red pepper strips 1 tbsp. red wine vinegar Directions: -Heat oil in saucepot. -Add carrots, celery, onion, parsley and garlic and cook until tender. -[MASK] -Heat to a boil. -Cover and cook 15 min. -or until pasta is done. -Add peppers and vinegar. -Heat through.

tgt: Add broth, beans, tomatoes and pasta.


src: Feta Cheese with Egg Ingredients: 1 clove garlic 1 teaspoon ground coriander 2 tablespoons extra-virgin olive oil 2 slices feta cheese 1 egg 1 tomato, chopped (optional) Directions: -[MASK] -Lay the slices of cheese on top and leave over medium heat until the cheese begins to fall apart. -Break open the egg over it, and if you like, surround with chopped tomato. -Cook until the egg is done to your lik

In [ ]:
print(masking30_src)

In [ ]:
print(masking30_tgt)

In [ ]:
masking30_tgt

In [ ]:
masking30_tgt